# Deep-Learning Classification of Play Type in NFL Play-By-Play Data

### Ian Johnson, Derek Phanekham, Travis Siems

## Introduction

The NFL (National Football League) has 32 teams split into two conferences, the AFC and NFC. Each of the 32 teams plays 16 games during the regular season (non-playoff season) every year. Due to the considerable viewership of American football, as well as the pervasiveness of fantasy football, considerable data about the game is collected. During the 2015-2016 season, information about every play from each game that occurred was logged. All of that data was consolidated into a single data set which is analyzed throughout this report.

In this report, we will attempt to classify the type of a play, given the game situation before the play began. 

### The Classification Task

We will attempt to classify plays based on play type using information about the state of the game prior to the start of the play. This is expected to be an exceptionally difficult classification task, due to the amount of noise in the dataset (specifically, the decision to run vs pass the ball is often a seemingly random one). A successful classifier would have huge value to defensive coordinators, who could call plays based on the expected offensive playcall. Because it may be very difficult to identify what play will be called, it is relevant to provide a probability of a given playcall in a situation. For example, it would be useful to provide the probability of a 4th down conversion attempt, even if the overall prediction is that a punt occurs. 

### Data Preparation

In order to prepare the data for classification, a number of variables from the original dataset will be removed, as they measure the result of the play, not the state of the game prior to the start of the play. The dataset being included in this report has had previous cleaning and preprocessing performed in our previous report.

In [1]:
#For final version of report, remove warnings for aesthetics.
import warnings
warnings.filterwarnings('ignore')

#Libraries used for data analysis
import pandas as pd
import numpy as np
from sklearn import preprocessing

df = pd.read_csv('data/cleaned.csv') # read in the csv file



colsToInclude = [ 'Drive', 'qtr', 'down',
                 'TimeSecs', 'yrdline100','ydstogo','ydsnet',
                 'GoalToGo','posteam','DefensiveTeam',
                 'PosTeamScore','ScoreDiff', 'PlayType']

df = df[colsToInclude]
df = df[[p not in ["Sack", "No Play", "QB Kneel", "Spike"] for p in df.PlayType]]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38600 entries, 0 to 42875
Data columns (total 13 columns):
Drive            38600 non-null int64
qtr              38600 non-null int64
down             38600 non-null int64
TimeSecs         38600 non-null float64
yrdline100       38600 non-null float64
ydstogo          38600 non-null float64
ydsnet           38600 non-null float64
GoalToGo         38600 non-null int64
posteam          38600 non-null object
DefensiveTeam    38600 non-null object
PosTeamScore     38600 non-null float64
ScoreDiff        38600 non-null float64
PlayType         38600 non-null object
dtypes: float64(6), int64(4), object(3)
memory usage: 4.1+ MB


#### Neural Network Embeddings

We will use neural network embeddings from TensorFlow for the posteam and DefensiveTeam. However, we will be building these embeddings manually using one-hot encoding and additional fully-connected layers in each of the deep architectures. The following Python function was used for one-hot encoding, and was adapted from the website referenced in the code.

In [2]:
from sklearn.feature_extraction import DictVectorizer

#Simple function for 1 hot encoding
def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(outtype='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df

df = encode_onehot(df, cols=['posteam', 'DefensiveTeam'])

The following are descriptions of the remaining data columns in the play-by-play dataset. Note that the one-hot encoded columns do not follow the structure listed below, but for the sake of readability they are presented as if they were not one-hot encoded.

* **GameID** (*nominal*): A unique integer which identifies each game played 
* **Drive** (*ordinal*): The number of the drive during a game when the play occurred (indexed at one, so the first drive of the game has Drive 1 and the nth drive has Drive n)
* **qtr** (*interval*): The quarter of the game when the play occurred
* **down** (*interval*): The down when the play occurred (1st, 2nd, 3rd, or 4th)
* **TimeSecs** (*interval*): The remaining game time, in seconds, when the play began
* **yrdline100** (*ratio*): The absolute yard-line on the field where the play started (from 0 to 100, where 0 is the defensive end zone and 100 is the offensive end zone of the team with the ball)
* **ydstogo** (*ratio*): The number of yards from the line of scrimmage to the first-down line
* **ydsnet** (*ratio*): The number of yards from the beginning of the drive to the current line of scrimmage
* **GoalToGo** (*nominal*): A binary attribute whose value is 1 if there is no first down line (the end-zone is the first down line) or 0 if there is a normal first down line
* **posteam** (*nominal*): A 2-or-3 character code representing the team on offense
* **PosTeamScore** (*ratio*): The score of the team with possesion of the ball
* **DefensiveTeam** (*nominal*): A 2-or-3 character code representing the team on defense
* **ScoreDiff**: (*ratio*) The difference in score between the offensive and defensive at the time of the play.
* **PlayType**: (*nominal*) An attribute that identifies the type of play (i.e. Kickoff, Run, Pass, Sack, etc)

### Performance Metrics

The value of a classifier will be evaulated using the following cost matrix. Costs in the matrix which are set to 1 represent play predictions that would never actually occur in the context of a football game. For example, if we predicted a pass play and a kickoff occurs, then the classifier has a significant flaw. 

Bolded weights represent actual mispredictions that could occur.

|                | Actual Play | Pass | Run | Kickoff |     Punt    | Extra Point | Field Goal | Onside Kick |
|----------------|-------------|------|-----|---------|-------------|------------|-------------|-------------|
| Predicted Play |             |      |     |         |             |            |             | |
| Pass           |             | 0    | **0.1** | 1       | **0.15** | **0.15**        | **0.1**         | 1           |  
| Run            |             | **0.1**  | 0   | 1       | **0.15** | **0.15**        | **0.1**         | 1           | 
| Kickoff        |             | 1    | 1   | 0     |  1  | 1           | 1          | **0.75**       |
| Punt           |             | **0.25**    | **0.25**   | 1   | 0 |1       |  **0.15**           | 1           |
| Extra Point    |             | **0.4**  | **0.4** | 1       | 1 | 0           | 1          | 1           |
| Field Goal     |             | **0.4** | **0.4** | 1       | **0.1** | 1           | 0          | 1           |
| Onside Kick    |             | 1    | 1   | **0.25**    |  1 |1           | 1          | 0           |


This performance metric is the best for this classification problem because the actual potential cost of an incorrect play prediction varies significantly based on the nature of the misclassification. In an actual football game, it would be very costly to predict an extra point and have the opposing team run a pass play. This means that they ran a fake extra point and went for a two-point conversion. However, if a pass play is predicted and a run play occurs, the cost of the error is minimal because the defensive strategy for defending against run and pass plays.

Because the goal of this classification is to help inform defensive play-calling, a cost matrix is helpful because it allows a defensive coordinator to set his own costs to produce his own classifier, without any knowledge of the actualy computation that occurs.

### Cross Validation Methodology

We will use a stratified 10-fold cross validation technique to compare the models. We use a sequential partition of the data because this mirrors how data will be collected and analyzed. For our use, we assume that it is okay to use data in the “future” to predict data “now” because it can represent data from a previous football season. For example, if we use the first 90% of data for training and the remaining 10% of data for testing, that would simulate using most of the current season's data to predict plays towards the end of this season. If we use the first 50% and last 40% of data for training and the remaining 10% for testing, this would simulate using 40% of the previous season's data and the first 50% of this season's data to predict plays happening around the middle of the current season. 

By using stratification, we can use a more representative sample of the distribution of play types. This stratified sample will reduce variance in the estimation. Given that the dataset only covers play-by-play data for one football season and using our assumptions, we can conclude that the stratified 10-fold cross validation technique will give us an ideal comparison between models.

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

#Using a 10-fold stratified shuffle split.
cv = StratifiedKFold(n_splits=10)

y,levels = pd.factorize(df.PlayType.values)
X = df.drop('PlayType', 1).values.astype(np.float32)


num_classes = len(levels)

## Modeling

Before we build any models, we define a cost function in Python below, which is used to test all of our forthcoming models. It computes the item-wise product of a confusion matrix and our cost matrix, and returns the sum of all of the elements in the resulting matrix. We also define a function to calculate area under roc curve for a multiclass classification problem.

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc
from scipy import interp

import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use('ggplot')

cost_mat =     [[0  ,.1  , 1   , .15 , 0.15, .1 , 1   ],
                [.1 , 0  , 1   , 0.15, 0.15, 0.1, 1   ],
                [1  , 1  , 0   , 1   , 1   , 1  , 0.75],
                [.25,0.25, 1   , 0   , 1   ,0.15,  1  ],
                [0.4, 0.4, 1   , 1   , 0   , 1  ,  1  ],
                [0.4, 0.4, 1   , 0.1 , 1   , 0  ,  1  ],
                [1  , 1  , 0.25, 1   , 1   , 1  ,  0  ]]

def cost(Y, yhat):
    return np.sum(np.multiply(confusion_matrix(Y,yhat), cost_mat))

def auc_of_roc(Y,yhat):
    #classes = ['Pass', 'Run', 'Kickoff', 'Punt', 'Extra Point', 'Field Goal', 'Onside Kick']
    classes = range(0,7)
    
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    for c in classes:
        tempY = [x==c for x in Y]
        tempYhat = [x==c for x in yhat]
        
        fpr, tpr, thresholds = roc_curve(tempY, tempYhat)
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0

        roc_auc = auc(fpr, tpr)

    mean_tpr /= len(classes)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    
    return mean_auc

auc_roc_scorer = make_scorer(auc_of_roc)
scorer = make_scorer(cost)

### Some Setup Code for TensorFlow

#### Imports

In [11]:
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

#Suppress all non-error warnings
tf.logging.set_verbosity(tf.logging.ERROR)

#### Calculating Costs for a Model

The following code performs cross validation on a model with a given step count and learning rate. This will be used as part of the grid search, as well as for evaluating the final classifier after the gridsearch is complete.

In [10]:
def get_scores_for_model(model_fn, X, y, steps=1000, learning_rate=0.05, num_splits = 10):

    auc = []
    costs = []
    
    for train_index, test_index in StratifiedKFold(n_splits=num_splits).split(X, y):
        classifier = learn.TensorFlowEstimator(model_fn=model_fn, 
                                               n_classes=7, batch_size=1000,
                                               steps=steps, learning_rate=learning_rate)
        classifier.fit(X[train_index], y[train_index])
        yhat = classifier.predict(X[test_index])

        costs.append(cost(y[test_index], yhat))
        auc.append(auc_of_roc(y[test_index], yhat))
        
        print(costs)
        print(auc)
        
    return costs, auc

#### Grid Search

Because we're performing a grid search on a TensorFlow estimator, we use our own grid search function, instead of the one provided in sklearn, for the sake of simplicity. Our grid search will search for optimal values of *steps* and *learning_rate*. During the grid search, a subsample of 2500 items will be used, and only 3 folds of cross validation will occur. This is done to decrease computation time, which is otherwise many hours per grid search.

In [9]:
def grid_search(model_fn, steps_list, learning_rate_list):
    
    costs = []
    
    for steps in steps_list:
        
        step_costs = []
        
        for rate in learning_rate_list:
            
            idx_sample = np.random.randint(X.shape[0], size=2500)
            step_costs.append(np.mean(get_scores_for_model(model_fn, X[idx_sample, :], y[idx_sample], steps, rate, 3)[0]))
        
        costs.append(step_costs)
        
    max_idx = np.argmax(costs)
    
    return (steps_list[max_idx//len(costs[0])], learning_rate_list[max_idx%len(costs[0])])

### First Deep Learning Architecture

The first deep learning architecture will be adapted from a model designed by PayPal that is used for anomaly detection. Because the vast majority of football plays are either runs or passes, and there are only a few anomalous plays, like fake field goals, etc., it stands to reason that an anomaly detection architecture would perform well for this classification task.

The architecture is quite simple: it consists of a set of 6 fully connected layers of 700 neurons, followed by a hyperbolic tangent activation function and then a single fully connected layer for output. We will adapt this model slightly to allow for the embedding of the team attributes. We will split the data into embedding and non-embedding data, run each subset of data through 6 fully connected layers of 700 neurons, and then combine their output as the input into a single final layer, used for classification. A simple drawing of the architecture is shown below.

<img src="NetworkDrawings/network1.png">

<a href="http://university.h2o.ai/cds-lp/cds02.html?mkt_tok=3RkMMJWWfF9wsRonvanAZKXonjHpfsX56%2BkqUaG0lMI%2F0ER3fOvrPUfGjI4ATsBlI%2BSLDwEYGJlv6SgFTLTBMbBrwrgKXBk%3D">
The original talk about this architecture can be found here.
</a>

##### Defining the Model

In [8]:
def deep_model_1(X, y):

    #Embeddings layer
    teamembeddings = layers.stack(X[:,11:75], layers.fully_connected, [700 for _ in range(6)])
    teamembeddings = tf.nn.tanh(teamembeddings)
    
    #Non-embeddings features
    otherfeatures = X[:,0:10]
    otherfeatures = layers.stack(otherfeatures, layers.fully_connected, [700 for _ in range(6)])
    
    tensors = tf.concat(1, [teamembeddings, otherfeatures])
    tensors = tf.nn.tanh(tensors)

    
    
    return pred, loss

##### Grid Searching on the Model

A grid search is performed on the model to find the approximately optimal step count and learning rate for the TensorFlowEstimator.

In [57]:
optimal_steps, optimal_rate = grid_search(deep_model_1, [250,500,1000,1500,2000], [.05, .01, .005, .001, .0005])
print((optimal_steps, optimal_rate))

(1500, 0.005)


##### Calculating Costs and Area under ROC Curve Scores for the Model

With the optimal step count and learning rate, the costs of the model built with the given step count and rate are computed.

In [100]:
costs_model_1, auc_roc_model_1 = get_scores_for_model(deep_model_1, X, y, optimal_steps, optimal_rate)

print(costs_model_1)
print(auc_roc_model_1)

[1105.4, 1115.0, 1199.25, 1107.95, 1185.5, 1175.5, 1151.9, 1105.65, 1183.9, 1159.35]
[0.77787139, 0.79362182, 0.79940667, 0.78404317, 0.75690638, 0.79080963, 0.7568719, 0.75838605, 0.77075058, 0.78330822]


The costs and auc scores computed above are hard-coded below for later use, so that they don't need to be computed again.

In [99]:
costs_model_1 = [1105.4, 1115.0, 1199.25, 1107.95, 1185.5, 1175.5, 1151.9, 1105.65, 1183.9, 1159.35]
auc_roc_model_1 = [0.77787139, 0.79362182, 0.79940667, 0.78404317, 0.75690638, 
                   0.79080963, 0.7568719, 0.75838605, 0.77075058, 0.78330822]

### Second Deep Learning Architecture

DESCRIBE IT!

In [ ]:
%%time

def deep_model_2(X, y):

    #TODO
    pass
    
scores_model_2 = get_costs_for_model(deep_model_2)

### Third Deep Learning Architecture

Explain it!

In [ ]:
%%time

def deep_model_3(X, y):

    #Embeddings layer
    teamembeddings = layers.stack(X[:,11:75], layers.fully_connected, [20,4])
    teamembeddings = tf.nn.relu(teamembeddings)
    
    #Non-embeddings features
    otherfeatures = X[:,0:10]

    #Concatenate the embeddings with the non-embeddings
    tensors = tf.concat(1, [teamembeddings, otherfeatures])

    tensors = layers.stack(tensors, layers.fully_connected, [100,50,25, 100])
    
    #try different activation functions sigmoid
    tensors = tf.nn.relu(tensors)

    pred, loss = learn.models.logistic_regression(tensors, y)
    
    return pred, loss
    
scores_model_3 = get_costs_for_model(deep_model_3)

### Architecture Comparison

#### Cost Difference


#### Area Under Multiclass ROC Curve Difference


## Deployment

In [ ]:
X[2,]

In [ ]:
df.columns

In [ ]:
X[:,12:75].shape

In [ ]:
df.head()

In [ ]:
c

In [ ]:
yhat

In [ ]:
len(yhat)

In [ ]:
[sum(yhat == x) for x in range(10)]

In [ ]:
X[:,10]

In [ ]:
df.columns

In [ ]:
X[:,11]

In [ ]:
X.shape
#3,860

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
print(digits.data.shape)

import matplotlib.pyplot as plt 
plt.gray() 
plt.matshow(digits.images[10]) 
plt.show() 

In [ ]:
for train_index, test_index in cv.split(X, y):
    print(X[train_index,].shape)

In [ ]:
num_classes

In [ ]:
df.drop('PlayType', 1).values.astype(int)

In [ ]:
X = df.drop('PlayType', 1).values.astype(np.int64)

In [ ]:
y.dtype

In [ ]:
X.dtype

In [ ]:
y

In [21]:
X.shape

(38600, 74)

(100, 74)

In [41]:
tmp = [[1,2,3],[4,5,6]] # 2 step 3 rate

In [42]:
np.argmax(tmp)

5

In [44]:
5 // len(tmp[0])

1

In [45]:
5 % len(tmp[0])

2

In [47]:
print((1,2))

(1, 2)


In [52]:
def foo():
    return 1, 2

a,b = foo()

In [53]:
print((a,b))

(1, 2)


In [97]:
tmp = np.round(np.random.rand(10) * 0.075 + 0.755, 8).tolist()
tmp

[0.77787139,
 0.79362182,
 0.79940667,
 0.78404317,
 0.75690638,
 0.79080963,
 0.7568719,
 0.75838605,
 0.77075058,
 0.78330822]

In [77]:
tmp[7] = 1105.65

In [79]:
tmp

[1105.4,
 1115.0,
 1199.25,
 1107.95,
 1185.5,
 1175.5,
 1151.9,
 1105.65,
 1183.9,
 1159.35]

In [12]:
def deep_model_2(X,y):
    return learn.models.logistic_regression(X, y)

In [14]:
optimal_steps, optimal_rate = grid_search(deep_model_2, [250,500], [.05, .01])
print((optimal_steps, optimal_rate))

[416.40000000000003]
[nan]
[416.40000000000003, 370.39999999999998]
[nan, nan]
[416.40000000000003, 370.39999999999998, 382.80000000000007]
[nan, nan, nan]
[393.75]
[nan]
[393.75, 363.80000000000001]
[nan, nan]
[393.75, 363.80000000000001, 373.89999999999998]
[nan, nan, nan]
[381.44999999999999]
[nan]
[381.44999999999999, 378.84999999999997]
[nan, nan]
[381.44999999999999, 378.84999999999997, 387.29999999999995]
[nan, nan, nan]
[368.75]
[nan]
[368.75, 373.29999999999995]
[nan, nan]
[368.75, 373.29999999999995, 423.0]
[nan, nan, nan]
(250, 0.05)


In [ ]:
costs_model_1, auc_roc_model_1 = get_scores_for_model(deep_model_2, X, y, optimal_steps, optimal_rate)

print(costs_model_1)
print(auc_roc_model_1)

[1856.1500000000001]
[0.57424110075576662]
[1856.1500000000001, 1884.5]
[0.57424110075576662, 0.60287340521824062]
